In [40]:
import sqlite3

In [41]:
 conn = sqlite3.connect(":memory:")  # in-memory DB (use "mydb.db" for file)
 cursor = conn.cursor()

In [42]:
cursor.execute("""
 CREATE TABLE Departments (
    dept_id INTEGER PRIMARY KEY AUTOINCREMENT,
    dept_name TEXT NOT NULL,
    hod_name TEXT
    )
    """)
cursor.execute("""
CREATE TABLE Students (
    student_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    class TEXT NOT NULL,
    dept_id INT,
    FOREIGN KEY (dept_id) REFERENCES Departments(dept_id)
    )
    """)
cursor.execute("""
CREATE TABLE Subjects (
    subject_id INTEGER PRIMARY KEY AUTOINCREMENT,
    subject_name TEXT NOT NULL,
    dept_id INT,
    FOREIGN KEY (dept_id) REFERENCES Departments(dept_id)
    )
    """)
cursor.execute("""
CREATE TABLE Marks (
    mark_id INTEGER PRIMARY KEY AUTOINCREMENT,
    student_id INTEGER,
    subject_id INTEGER,
    marks INT, 
    FOREIGN KEY (student_id) REFERENCES Students(student_id),
    FOREIGN KEY (subject_id) REFERENCES Subjects(subject_id)
    )
    """)
departments = [
    ("Math", "HOD 1"),
    ("Science", "HOD 2")
]
cursor.executemany("INSERT INTO Departments (dept_name, hod_name) VALUES (?, ?)", departments)

students = [
    ("Joe", "Class 1", 1),
    ("Bob", "Class 1", 1),
    ("Tom", "Class 2", 2),
    ("Amy", "Class 2", 2),
    ("Sue", "Class 1", 1)
]
cursor.executemany("INSERT INTO Students (name, class, dept_id) VALUES (?, ?, ?)", students)

subjects = [
    ("Algebra", 1),
    ("Geometry", 1),
    ("Biology", 2),
    ("Chemistry", 2)
]
cursor.executemany("INSERT INTO Subjects (subject_name, dept_id) VALUES (?, ?)", subjects)

# Each student gets marks in all 4 subjects
marks_data = [
    (1, 1, 70), (1, 2, 75), (1, 3, 80), (1, 4, 85),
    (2, 1, 60), (2, 2, 65), (2, 3, 70), (2, 4, 75),
    (3, 1, 80), (3, 2, 85), (3, 3, 90), (3, 4, 95),
    (4, 1, 55), (4, 2, 60), (4, 3, 65), (4, 4, 70),
    (5, 1, 90), (5, 2, 85), (5, 3, 80), (5, 4, 75)
]
cursor.executemany("INSERT INTO Marks (student_id, subject_id, marks) VALUES (?, ?, ?)", marks_data)

conn.commit()

In [60]:
def add_student(conn, name, class_, dept_id):
    conn.execute(
        """
        INSERT INTO Students (name, class, dept_id)
        VALUES (?, ?, ?)
        """,
        (name, class_, dept_id)
    )
    conn.commit()

def add_subject(conn, subject_name, dept_id):
    conn.execute(
        """
        INSERT INTO Subjects (subject_name, dept_id)
        VALUES (?, ?)
        """,
        (subject_name, dept_id)
    )
    conn.commit()

def add_marks(conn, student_id, subject_id, marks):
    conn.execute(
        """
        INSERT INTO Marks (student_id, subject_id, marks)
        VALUES (?, ?, ?)
        """,
        (student_id, subject_id, marks)
    )
    conn.commit()

def show_students(conn):
    return conn.execute(
        """
        SELECT Students.student_id, Students.name, Students.class, Departments.dept_name
        FROM Students
        JOIN Departments ON Students.dept_id = Departments.dept_id
        """
    ).fetchall()

def show_subjects(conn):
    return conn.execute(
        """
        SELECT Subjects.subject_id, Subjects.subject_name, Departments.dept_name
        FROM Subjects
        JOIN Departments ON Subjects.dept_id = Departments.dept_id
        """
    ).fetchall()

def show_marks(conn):
    return conn.execute("""
        SELECT Marks.mark_id, Students.name, Subjects.subject_name, Marks.marks 
        FROM Marks 
        JOIN Students ON Marks.student_id = Students.student_id 
        JOIN Subjects ON Marks.subject_id = Subjects.subject_id
    """).fetchall()

def student_report(conn, student_id):
    return conn.execute("""
        SELECT Subjects.subject_name, Marks.marks 
        FROM Marks 
        JOIN Subjects ON Marks.subject_id = Subjects.subject_id 
        WHERE Marks.student_id = ?
    """, (student_id,)).fetchall()

def toppers(conn):
    subject_toppers = conn.execute("""
        SELECT Subjects.subject_name, Students.name, MAX(Marks.marks) 
        FROM Marks 
        JOIN Students ON Marks.student_id = Students.student_id 
        JOIN Subjects ON Marks.subject_id = Subjects.subject_id 
        GROUP BY Marks.subject_id
    """).fetchall()

    overall_topper = conn.execute("""
        SELECT Students.name, SUM(Marks.marks) 
        FROM Marks 
        JOIN Students ON Marks.student_id = Students.student_id 
        GROUP BY Marks.student_id 
        ORDER BY SUM(Marks.marks) DESC 
        LIMIT 1
    """).fetchone()

    return subject_toppers, overall_topper

def department_summary(conn):
    return conn.execute("""
        SELECT Departments.dept_name, AVG(Marks.marks) 
        FROM Marks 
        JOIN Students ON Marks.student_id = Students.student_id 
        JOIN Departments ON Students.dept_id = Departments.dept_id 
        GROUP BY Departments.dept_id
    """).fetchall()

def unique_subjects(conn):
    rows = conn.execute("""
        SELECT DISTINCT subject_name 
        FROM Subjects
    """).fetchall()
    return {r[0] for r in rows}


In [58]:
show_subjects(conn)

[(1, 'Algebra', 'Math'),
 (2, 'Geometry', 'Math'),
 (3, 'Biology', 'Science'),
 (4, 'Chemistry', 'Science')]

In [59]:
show_students(conn)

[(1, 'Joe', 'Class 1', 'Math'),
 (2, 'Bob', 'Class 1', 'Math'),
 (3, 'Tom', 'Class 2', 'Science'),
 (4, 'Amy', 'Class 2', 'Science'),
 (5, 'Sue', 'Class 1', 'Math')]

In [61]:
add_student(conn, "Joe", "Class 1", 1)

In [62]:
show_students(conn)

[(1, 'Joe', 'Class 1', 'Math'),
 (2, 'Bob', 'Class 1', 'Math'),
 (3, 'Tom', 'Class 2', 'Science'),
 (4, 'Amy', 'Class 2', 'Science'),
 (5, 'Sue', 'Class 1', 'Math'),
 (6, 'Joe', 'Class 1', 'Math')]